In [1]:
# Ganesh Arvapalli
# NeuroData - 11/14/18

import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup
import requests
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab as pl
import time
import numpy as np
import src.features.summary
import tempfile
import pickle

ModuleNotFoundError: No module named 'networkx'

In [ ]:
# Now using the GraphStats class, see documentation for details

def generate_stats(g):
    return Stats(g).return_stats()

In [ ]:
f = open("sub-0025427_ses-10_dwi_JHU.gpickle", "rb")
tmp = pickle.load(f)
print("tmp:", tmp)
print(tmp is None)
print(nx.info(tmp))

In [ ]:
g = nx.read_gpickle("sub-0025427_ses-10_dwi_JHU.gpickle")
print("g:", g)
print(g is None)
print(nx.info(g))

In [ ]:
http = urllib3.PoolManager()

subjects=['0025427', '0025428', '0025429', '0025430', '0025431', '0025432', '0025433',
       '0025434', '0025435', '0025436', '0025437', '0025438', '0025439', '0025440',
       '0025441', '0025442', '0025443', '0025444', '0025445', '0025446', '0025447',
       '0025448', '0025449', '0025450', '0025451', '0025452', '0025453', '0025454',
       '0025455', '0025456']

data_matrix = []
count = 0
for j in subjects:
    for i in range(1,11):
        r = http.request("GET", "https://mrneurodata.s3.amazonaws.com/data/HNU1/ndmg_0-0-48/graphs/JHU/sub-"+ j +"_ses-" + str(i) + "_dwi_JHU.gpickle")
        if r.status == 200:
            try:
                f = open("tempfile.gpickle", "wb")
                f.write(r.data)
                f.close()
                f = open("tempfile.gpickle", "rb")
                tmp = pickle.load(r.data)
                print(tmp)
                g = nx.read_gpickle(tmp)
                f.close()
                
                count += 1

                stats = generate_stats(g)
                data_matrix = np.vstack([data_matrix, stats])
#             except ValueError:
#                 print("ValueError encountered!")
# #                 print(filedata)
#                 print(j)
#                 continue
            except TypeError:
                print("TypeError encountered!")
                for i in stats:
                    print(type(i))
                break

# data_matrix = np.delete(data_matrix, 0, axis=0)


In [ ]:
print("Done collecting data")
print("Data shape:", data_matrix.shape)
# print(data_matrix)
print("Total number of graphs read:", count)
print("Number of features:", data_matrix.shape[1])

In [ ]:
# SKLearn Agglomerative Clustering - Ward

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import math

best_ss = []
plt.figure()
plt.title('Num Clusters vs. Silhouette Score (Agg, Ward), w/ N components')
for i in range(2,6):
    x = range(2,15)
    y = []
    for j in x:
    pca = PCA(n_components=i, svd_solver='full')
    X = pca.fit_transform(data_matrix)
    clustering = AgglomerativeClustering(n_clusters=j).fit(X)
    # print(clustering.labels_)
    y.append(silhouette_score(X, clustering.labels_, metric='sqeuclidean'))
  
    plt.xlabel('# of clusters')
    plt.ylabel('Silhouette Score')
    plt.plot(x, y)

    best_ss.append(max(y))

plt.legend(range(2,6))
plt.show()
  
plt.figure()
plt.title('Best Silhouette Score per Num Components: ' + str(round(max(best_ss), 3)))
plt.xlabel('# of components')
plt.ylabel('Max Silhouette Score')
plt.plot(range(2,6), best_ss)
plt.show()

# Best case scenario
pca = PCA(n_components=2, svd_solver='full')
X = pca.fit_transform(data_matrix)
aggclust = AgglomerativeClustering(n_clusters=4).fit(X)
# print(clustering.labels_)
print(silhouette_score(X, aggclust.labels_, metric='sqeuclidean'))

plt.figure()
plt.title("Agglomerative Ward-Linked Clustering Results on PCA 2D (4 clusters)")
plt.xlabel("Eigenvalue 1")
plt.ylabel("Eigenvalue 2")
plt.scatter(X[:, 0], X[:, 1], c=aggclust.labels_,cmap=plt.cm.jet)
plt.show()


In [ ]:
# SKLearn Agglomerative Clutering - Complete

best_ss = []
plt.figure()
plt.title('Num Clusters vs. Silhouette Score (Agg, Complete), w/ N components')
for i in range(2,6):
    x = range(2,15)
    y = []
    for j in x:
        pca = PCA(n_components=i, svd_solver='full')
        X = pca.fit_transform(data_matrix)
        clustering = AgglomerativeClustering(n_clusters=j, linkage="complete").fit(X)
        # print(clustering.labels_)
        y.append(silhouette_score(X, clustering.labels_, metric='sqeuclidean'))
  
    plt.xlabel('# of clusters')
    plt.ylabel('Silhouette Score')
    plt.plot(x, y)

    best_ss.append(max(y))

plt.legend(range(2,6))
plt.show()
  
plt.figure()
plt.title('Best Silhouette Score per Num Components: ' + str(round(max(best_ss), 3)))
plt.xlabel('# of components')
plt.ylabel('Max Silhouette Score')
plt.plot(range(2,6), best_ss)
plt.show()

# Best case scenario
pca = PCA(n_components=2, svd_solver='full')
X = pca.fit_transform(data_matrix)
aggclust2 = AgglomerativeClustering(n_clusters=2, linkage="complete").fit(X)
# print(clustering.labels_)
print(silhouette_score(X, aggclust2.labels_, metric='sqeuclidean'))

plt.figure()
plt.title("Agglomerative Complete-Linked Clustering Results on PCA 2D (2 clusters)")
plt.xlabel("Eigenvalue 1")
plt.ylabel("Eigenvalue 2")
plt.scatter(X[:, 0], X[:, 1], c=aggclust2.labels_,cmap=plt.cm.jet)
plt.show()

In [ ]:
# SKLearn Agglomerative Clustering - Average

best_ss = []
plt.figure()
plt.title('Num Clusters vs. Silhouette Score (Agg, Average), w/ N components')
for i in range(2,6):
    x = range(2,15)
    y = []
    for j in x:
        pca = PCA(n_components=i, svd_solver='full')
        X = pca.fit_transform(data_matrix)
        clustering = AgglomerativeClustering(n_clusters=j, linkage="average").fit(X)
        # print(clustering.labels_)
        y.append(silhouette_score(X, clustering.labels_, metric='sqeuclidean'))
  
    plt.xlabel('# of clusters')
    plt.ylabel('Silhouette Score')
    plt.plot(x, y)

    best_ss.append(max(y))

plt.legend(range(2,6))
plt.show()
  
plt.figure()
plt.title('Best Silhouette Score per Num Components: ' + str(round(max(best_ss), 3)))
plt.xlabel('# of components')
plt.ylabel('Max Silhouette Score')
plt.plot(range(2,6), best_ss)
plt.show()

# Best case scenario
pca = PCA(n_components=2, svd_solver='full')
X = pca.fit_transform(data_matrix)
aggclust2 = AgglomerativeClustering(n_clusters=2, linkage="average").fit(X)
# print(clustering.labels_)
print(silhouette_score(X, aggclust2.labels_, metric='sqeuclidean'))

plt.figure()
plt.title("Agglomerative Average-Linked Clustering Results on PCA 2D (2 clusters)")
plt.xlabel("Eigenvalue 1")
plt.ylabel("Eigenvalue 2")
plt.scatter(X[:, 0], X[:, 1], c=aggclust2.labels_,cmap=plt.cm.jet)
plt.show()

In [ ]:
# SKLearn KMeans Clustering

from sklearn.cluster import KMeans

best_ss = []
plt.figure()
plt.title('Num Clusters vs. Silhouette Score (K-Means) w/ N components')
for i in range(2,6):
    x = range(2,15)
    y = []
    for j in x:
        pca = PCA(n_components=i, svd_solver='full')
        X = pca.fit_transform(data_matrix)
        clustering = KMeans(n_clusters=j).fit(X)
        # print(clustering.labels_)
        y.append(silhouette_score(X, clustering.labels_, metric='sqeuclidean'))
  
    plt.xlabel('# of clusters')
    plt.ylabel('Silhouette Score')
    plt.plot(x, y)

    best_ss.append(max(y))

plt.legend(range(2,6))
plt.show()
  
plt.figure()
plt.title('Best Silhouette Score per Num Components: ' + str(round(max(best_ss), 3)))
plt.xlabel('# of components')
plt.ylabel('Max Silhouette Score')
plt.plot(range(2,6), best_ss)
plt.show()

# Best case scenario
pca = PCA(n_components=2, svd_solver='full')
X = pca.fit_transform(data_matrix)
kmeans = KMeans(n_clusters=2).fit(X)
# print(clustering.labels_)
print(silhouette_score(X, kmeans.labels_, metric='sqeuclidean'))

plt.figure()
plt.title("K-Means Clustering Results on PCA 2D (2 clusters)")
plt.xlabel("Eigenvalue 1")
plt.ylabel("Eigenvalue 2")
plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_,cmap=plt.cm.jet)
plt.show()

In [ ]:
# SKLearn Spectral Clustering (Currently Faulty Implementation)

from sklearn.cluster import SpectralClustering

best_ss = []
plt.figure()
plt.title('Num Clusters vs. Silhouette Score (Spectral), w/ N components')
for i in range(2,6):
    x = range(2,15)
    y = []
    for j in x:
        pca = PCA(n_components=i, svd_solver='full')
        X = pca.fit_transform(data_matrix)
        clustering = SpectralClustering(n_clusters=j, assign_labels="discretize").fit(X)
        # print(clustering.labels_)
        y.append(silhouette_score(X, clustering.labels_, metric='sqeuclidean'))
  
    plt.xlabel('# of clusters')
    plt.ylabel('Silhouette Score')
    plt.plot(x, y)

    best_ss.append(max(y))

plt.legend(range(2,6))
plt.show()
  
plt.figure()
plt.title('Best Silhouette Score per Num Components: ' + str(round(max(best_ss), 3)))
plt.xlabel('# of components')
plt.ylabel('Max Silhouette Score')
plt.plot(range(2,6), best_ss)
plt.show()

# Best case scenario
pca = PCA(n_components=4, svd_solver='full')
# X = pca.fit_transform(nx.adjacency_matrix(g))
specclust = SpectralClustering(n_clusters=5, assign_labels="discretize").fit(nx.adjacency_matrix(g))
# print(clustering.labels_)
# print(silhouette_score(X, specclust.labels_, metric='sqeuclidean'))

plt.figure()
plt.title("Spectral Clustering Results on PCA 2D (5 clusters)")
plt.xlabel("Eigenvalue 1")
plt.ylabel("Eigenvalue 2")
plt.scatter(X[:, 0], X[:, 1], c=specclust.labels_,cmap=plt.cm.jet)
plt.show()